# Writing Data to Iceberg Table using Spark

In [ ]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import col
import os
from utils.utils import configure_spark, get_db_tables
# Define the path for the Iceberg warehouse
iceberg_warehouse_path = "iceberg_warehouse"  # Change this to your desired path




spark = configure_spark('minio','main')


local_table_name = "spark_catalog.default.employees"
df = spark.read.format("iceberg").table(local_table_name)



table_name = "nessie.employees"

schema = ', '.join([f'{field.name} {field.dataType.simpleString()}' for field in df.schema.fields])

def create_iceberg_table_if_not_exists(spark, table_name, df, partition_column):
    """Create an Iceberg table if it doesn't exist."""
    schema = ', '.join([f'`{field.name}` {field.dataType.simpleString()}' for field in df.schema.fields])
    create_table_query = f"""
        CREATE TABLE IF NOT EXISTS {table_name} (
            {schema}
        )
        USING iceberg
        PARTITIONED BY ({partition_column})
    """
    spark.sql(create_table_query)

# Write the DataFrame to the existing Iceberg table partitioned by the 'DATE' column
create_iceberg_table_if_not_exists(spark, table_name, df, 'companyName')
df.write.format("iceberg").mode("overwrite") \
    .save(table_name)

print(f"Data has been written to Iceberg table at {iceberg_warehouse_path}/default/{table_name}")

# Stop the Spark session
spark.stop()

# Processing Data with Spark and Iceberg

This script demonstrates how to process data using Spark and Iceberg. It includes functions to create Iceberg tables, query data from a Parquet file, and write data to both local and Nessie-managed Iceberg tables.

## Steps

1. **Configure Spark**: Set up a Spark session using a custom configuration function `configure_spark`.
2. **Create Iceberg Table**: Define a function to create an Iceberg table if it doesn't already exist.
3. **Query and Write to Local Table**: Define a function to read data from a Parquet file and write it to a local Iceberg table.
4. **Read Local and Write to Nessie**: Define a function to read data from a local Iceberg table and write it to a Nessie-managed Iceberg table.
5. **Process Data**: Define a function to process data by querying from a Parquet file and writing to both local and Nessie Iceberg tables.
6. **Execute Processing**: Set up paths and table names, then execute the data processing function.


In [ ]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import col
import os
from utils.utils import configure_spark

def create_iceberg_table_if_not_exists(spark, table_name, df, partition_column):
    """Create an Iceberg table if it doesn't exist."""
    schema = ', '.join([f'`{field.name}` {field.dataType.simpleString()}' for field in df.schema.fields])
    create_table_query = f"""
        CREATE TABLE IF NOT EXISTS {table_name} (
            {schema}
        )
        USING iceberg
        PARTITIONED BY ({partition_column})
    """
    spark.sql(create_table_query)

def query_and_write_to_local_table(spark, parquet_path, local_table_name, partition_column):
    """Query data from Nessie table for a given date and write it to the local Iceberg table."""
    df = spark.read.parquet(parquet_path)
    create_iceberg_table_if_not_exists(spark, local_table_name, df, partition_column)
    df.writeTo(local_table_name).append()

def read_local_and_write_to_nessie(spark, local_table_name, nessie_table_name, partition_column):
    """Read data from local Iceberg table and write it to the Nessie-managed Iceberg table."""
    local_df = spark.read.format("iceberg").table(local_table_name)
    create_iceberg_table_if_not_exists(spark, nessie_table_name, local_df, partition_column)
    local_df.writeTo(nessie_table_name).append()

def process_data(spark, parquet_path, local_table_name,nessie_table_name,partition_column):
    """Process a list of dates to query from Nessie and write to both local and Nessie Iceberg tables."""
  
    query_and_write_to_local_table(spark, parquet_path, local_table_name,partition_column)
    #read_local_and_write_to_nessie(spark, local_table_name, nessie_table_name, partition_column)



# Define the path for the Iceberg warehouse
iceberg_warehouse_path = "iceberg_warehouse"
os.makedirs(iceberg_warehouse_path, exist_ok=True)
# Initialize Spark session with the configured SparkConf
#spark = SparkSession.builder.config(conf=conf).getOrCreate()
spark = configure_spark('minio','main')

# Define the Nessie and local table names
parquet_path = 'scripts/python_go_backups/employees_go.parquet'
nessie_table_name = "nessie.employees"
local_table_name = "spark_catalog.default.employees"
partition_column = 'companyName'

process_data(spark, parquet_path, local_table_name,nessie_table_name, partition_column)


# Querying Employee Count by Company using Spark and Iceberg

This script demonstrates how to configure a Spark session, read an Iceberg table into a Spark DataFrame, register it as a temporary view, and perform a query to count employees grouped by company name.


In [ ]:
import pandas as pd
from utils.utils import configure_spark

local_table_name = "spark_catalog.default.employees"
spark = configure_spark('minio','main')

# Read the Iceberg table into a DataFrame
local_df = spark.read.format("iceberg").table(local_table_name)

# Register the DataFrame as a temporary view
local_df.createOrReplaceTempView("employees_view")

# Perform the query using Spark SQL
result_df = spark.sql("""
    SELECT companyName, COUNT(*) as employee_count
    FROM employees_view
    GROUP BY companyName
""")

# Show the results
result_df.toPandas()

## DuckDB and Iceberg Integration

This script demonstrates how to connect to a DuckDB database, install and load the Iceberg extension, and query an Iceberg table to save the results into a DuckDB table.

## Steps

1. **Connect to DuckDB**: Establish a connection to a DuckDB database file named `iceberg_data.duckdb`.
2. **Install and Load Iceberg Extension**: Install and load the Iceberg extension to enable querying Iceberg tables.
3. **Query Iceberg Table**: Execute a query to scan an Iceberg table located at `iceberg_warehouse/default/employees` and save the results into a DuckDB table named `employees`.

In [ ]:
import duckdb

# Connect to DuckDB
con = duckdb.connect('iceberg_data.duckdb')

# Install and load the Iceberg extension
con.execute("INSTALL iceberg;")
con.execute("LOAD iceberg;")

# Query the Iceberg table and save the results to a DuckDB table
con.execute("""
    CREATE TABLE employees AS
    SELECT *
    FROM iceberg_scan('iceberg_warehouse/default/employees', allow_moved_paths = true);
""")

# Querying Employee Count by Company

This script demonstrates how to connect to a DuckDB database, execute a query to get the count of employees grouped by company name, and convert the result into a pandas DataFrame.

## Steps

1. **Connect to DuckDB**: Establish a connection to a DuckDB database file named `iceberg_data.duckdb`.
2. **Execute Query**: Run a SQL query to count the number of employees grouped by `companyName` and order the results by `employee_count` in descending order.
3. **Convert to DataFrame**: Convert the query result into a pandas DataFrame for further analysis or manipulation.


In [ ]:
import duckdb
import pandas as pd

# Connect to DuckDB
con = duckdb.connect('iceberg_data.duckdb')

# Query to get the count of employees by companyName and convert it to a pandas DataFrame
df = con.execute("""
    SELECT companyName, COUNT(*) as employee_count
    FROM employees
    GROUP BY companyName ORDER BY employee_count DESC;
""").fetchdf()

df

In [ ]:
%pip install requests

In [3]:
import requests
1_241_512
url = "http://127.0.0.1:8000/create_companies"

# Define the payload
payload = {
    "request": {
        "size": 100
    },
    "s3_details": {
        "bucket_name": "upload",
        "database_name": "bronze-data"
    }
}

# Send the POST request
response = requests.post(url, json=payload)

# Print the response
print(response.status_code)
print(response.json())

200
{'message': 'Companies, employees, and departments created successfully', 'companies': [{'name': 'Walker LLC', 'industry': 'repurpose out-of-the-box vortals', 'employees': 3531, 'revenue': 157168626, 'location': 'North Kevin'}, {'name': 'Gilbert-Hanna', 'industry': 'evolve virtual functionalities', 'employees': 1380, 'revenue': 451544950, 'location': 'Tinaview'}, {'name': 'Thompson-Davis', 'industry': 'benchmark scalable experiences', 'employees': 7631, 'revenue': 401256914, 'location': 'New Donna'}, {'name': 'Dorsey Group', 'industry': 'expedite collaborative convergence', 'employees': 6082, 'revenue': 601388749, 'location': 'Jimenezshire'}, {'name': 'Thomas-Blankenship', 'industry': 'generate interactive infrastructures', 'employees': 5446, 'revenue': 690069536, 'location': 'Jennifertown'}, {'name': 'Ward, Gould and Rosario', 'industry': 'scale global web services', 'employees': 2866, 'revenue': 350485331, 'location': 'Jonesberg'}, {'name': 'Farrell PLC', 'industry': 'deliver eff